In [1]:
from neo4j import GraphDatabase, basic_auth

参考：https://neo4j.com/docs/python-manual/current/query-simple/

> neo4j 5.x 使用 driver.execute_query 执行 cql

连接到 neo4j

In [1]:
# 连接示例1
url="bolt://localhost:7687"
username='neo4j'
password='password'
driver = GraphDatabase.driver(url, auth=basic_auth(username, password))

In [ ]:
# 连接示例2
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "password")
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    try:
        # 业务逻辑
        pass
    except Exception as e:
        print(e)

创建节点

执行 merge 语句

In [4]:
# driver.execute_query 返回 records, summary, key
# records 是查询结果
# summary 是查询的统计信息
# key
summary = driver.execute_query(
    "MERGE (:Person {name: $name, age: $age})",
    name="Alice",
    age=25,
    # 最好显示指定数据库，使驱动程序可以更高效地工作
    database_="neo4j",
).summary
print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

Created 1 nodes in 16 ms.


执行查询语句

In [7]:
# driver.execute_query 返回 records, summary, key
# records 是查询结果
# summary 是查询的统计信息
# key
records, summary, keys = driver.execute_query("""
    MATCH (p:Person {name: $name})
    where p.age = $age
    RETURN p
    """,
    name="Alice",
    age=25,
    database_="neo4j",
)
# Loop through results and do something with them
for record in records:
    print(record.data())  # obtain record as dict

# 统计信息
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    # 查询语句
    query=summary.query,
    # 返回的记录数
    records_count=len(records),
    # 查询时间
    time=summary.result_available_after,
))

{'p': {'name': 'Alice', 'age': 25}}
The query `
    MATCH (p:Person {name: $name})
    where p.age = $age
    RETURN p
    ` returned 1 records in 26 ms.


查询语句-在字典中传递查询参数

In [8]:
parameters = {
    "name": "Alice",
    "age": 25
}
driver.execute_query(
    "MERGE (:Person {name: $name})",
    # 传递参数
    parameters_=parameters,
    database_="neo4j",
)
# Loop through results and do something with them
for record in records:
    print(record.data())  # obtain record as dict

{'p': {'name': 'Alice', 'age': 25}}


In [8]:
def runCql(cql, **kwargs):
    with driver.session() as session:
        data=session.run(cql, **kwargs)
    return data